In [51]:
import pandas as pd
import numpy as np
import numpy_financial as npf

from datetime import date, datetime, timedelta

# AIMS

1. Ratio de endeudamiento
2. Suma de cuota mensual monetaria por cada credito

## 1. Ratio de endeudamiento por cliente general

In [15]:
creditos = pd.read_csv('creditos.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
creditos.head()

,id_cliente,id_prestamo,monto,cuotas,deuda_actual,fecha_inicio,fecha_pago,fecha_pago.1,dias_atraso
0,1,10001,100,10,80,12/12/2022,12/12/2022,12/12/2022,0
1,1,10002,200,5,150,22/12/2022,22/12/2022,23/12/2022,1
2,2,10003,50,10,5,23/12/2022,23/12/2022,28/12/2022,5
3,2,10004,75,3,70,24/12/2022,24/12/2022,28/12/2022,4
4,2,10005,125,5,12,24/12/2022,24/12/2022,29/12/2022,5


In [16]:
clientes_agrupados = creditos.groupby('id_cliente').sum()
clientes_agrupados

C:\Users\Oscar\AppData\Local\Temp\ipykernel_7456\2360885637.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  clientes_agrupados = creditos.groupby('id_cliente').sum()


,id_prestamo,monto,cuotas,deuda_actual,dias_atraso
id_cliente,,,,,
1,40017,650,22,241,4
2,40019,650,28,109,18


In [17]:
clientes_agrupados['ratio_endeudamiento'] = clientes_agrupados['deuda_actual'] / clientes_agrupados['monto']
clientes_agrupados

,id_prestamo,monto,cuotas,deuda_actual,dias_atraso,ratio_endeudamiento
id_cliente,,,,,,
1,40017,650,22,241,4,0.370769
2,40019,650,28,109,18,0.167692


## 2. Suma de cuota mensual monetaria por cada credito

In [7]:
# funcion mes
def get_month(x):
    return datetime(x.year, x.month, 1)

In [8]:
creditos = pd.DataFrame({
    "Cliente": ["Juan", "Juan", "Ana", "Ana", "Pedro", "Pedro"],
    "Monto": [1000, 5000, 2000, 3000, 1500, 2000],
    "Cuotas": [12, 24, 6, 12, 8, 12],
    "Fecha": ["2022-01-01", "2022-02-01", "2022-01-01", "2022-03-01", "2022-01-01", "2022-02-01"]
})
creditos["Fecha"] = pd.to_datetime(creditos["Fecha"])
creditos['fecha_mes'] = creditos['Fecha'].apply(get_month)
creditos.set_index("fecha_mes", inplace=True)
creditos

,Cliente,Monto,Cuotas,Fecha
fecha_mes,,,,
2022-01-01,Juan,1000,12,2022-01-01
2022-02-01,Juan,5000,24,2022-02-01
2022-01-01,Ana,2000,6,2022-01-01
2022-03-01,Ana,3000,12,2022-03-01
2022-01-01,Pedro,1500,8,2022-01-01
2022-02-01,Pedro,2000,12,2022-02-01


In [9]:
creditos["Cuota Mensual"] = creditos["Monto"] / creditos["Cuotas"]
creditos

,Cliente,Monto,Cuotas,Fecha,Cuota Mensual
fecha_mes,,,,,
2022-01-01,Juan,1000,12,2022-01-01,83.333333
2022-02-01,Juan,5000,24,2022-02-01,208.333333
2022-01-01,Ana,2000,6,2022-01-01,333.333333
2022-03-01,Ana,3000,12,2022-03-01,250.000000
2022-01-01,Pedro,1500,8,2022-01-01,187.500000
2022-02-01,Pedro,2000,12,2022-02-01,166.666667


In [10]:
deuda_mensual = creditos.groupby([pd.Grouper(freq="M"), "Cliente"])["Cuota Mensual"].sum()
deuda_mensual

fecha_mes   Cliente
2022-01-31  Ana        333.333333
            Juan        83.333333
            Pedro      187.500000
2022-02-28  Juan       208.333333
            Pedro      166.666667
2022-03-31  Ana        250.000000
Name: Cuota Mensual, dtype: float64

In [11]:
deuda_mensual = deuda_mensual.reset_index()
deuda_mensual

,fecha_mes,Cliente,Cuota Mensual
0,2022-01-31,Ana,333.333333
1,2022-01-31,Juan,83.333333
2,2022-01-31,Pedro,187.500000
3,2022-02-28,Juan,208.333333
4,2022-02-28,Pedro,166.666667
5,2022-03-31,Ana,250.000000


## 3. 

In [27]:
creditos = pd.DataFrame({
    "client_id": ["Juan", "Juan", "Ana", "Ana", "Pedro", "Pedro"],
    "credit": [1000, 5000, 2000, 3000, 1500, 2000],
    "installment": [12, 24, 6, 12, 8, 12],
    "date": ["2022-01-01", "2022-02-01", "2022-01-01", "2022-03-01", "2022-01-01", "2022-02-01"]
})

creditos['installment_amount'] = round(creditos['credit'] / creditos['installment'],2)
creditos["date"] = pd.to_datetime(creditos["date"])
creditos['month'] = creditos['date'].dt.to_period('M')
creditos.head()

,client_id,credit,installment,date,installment_amount,month
0,Juan,1000,12,2022-01-01,83.33,2022-01
1,Juan,5000,24,2022-02-01,208.33,2022-02
2,Ana,2000,6,2022-01-01,333.33,2022-01
3,Ana,3000,12,2022-03-01,250.00,2022-03
4,Pedro,1500,8,2022-01-01,187.50,2022-01


In [28]:
cohort_data = creditos.groupby(['client_id', 'month'])['credit'].sum().reset_index()
cohort_data

,client_id,month,credit
0,Ana,2022-01,2000
1,Ana,2022-03,3000
2,Juan,2022-01,1000
3,Juan,2022-02,5000
4,Pedro,2022-01,1500
5,Pedro,2022-02,2000


In [29]:
pivoted_data = pd.pivot_table(cohort_data, values='credit', index='month', columns='client_id')
pivoted_data.fillna(0, inplace=True)
pivoted_data

client_id,Ana,Juan,Pedro
month,,,
2022-01,2000.0,1000.0,1500.0
2022-02,0.0,5000.0,2000.0
2022-03,3000.0,0.0,0.0


In [30]:
cumulative_data = pivoted_data.cumsum()
cumulative_data

client_id,Ana,Juan,Pedro
month,,,
2022-01,2000.0,1000.0,1500.0
2022-02,2000.0,6000.0,3500.0
2022-03,5000.0,6000.0,3500.0


In [32]:
cohort_data = creditos.groupby(['client_id', 'month', 'installment'])['installment_amount'].sum().reset_index()
cohort_data.head()

,client_id,month,installment,installment_amount
0,Ana,2022-01,6,333.33
1,Ana,2022-03,12,250.00
2,Juan,2022-01,12,83.33
3,Juan,2022-02,24,208.33
4,Pedro,2022-01,8,187.50


In [34]:
pivoted_data = pd.pivot_table(cohort_data, values='installment_amount', index='month', columns='client_id')
pivoted_data.fillna(0, inplace=True)
pivoted_data

client_id,Ana,Juan,Pedro
month,,,
2022-01,333.33,83.33,187.50
2022-02,0.00,208.33,166.67
2022-03,250.00,0.00,0.00


In [36]:
cumulative_data = pivoted_data.cumsum()
cumulative_data


client_id,Ana,Juan,Pedro
month,,,
2022-01,333.33,83.33,187.50
2022-02,333.33,291.66,354.17
2022-03,583.33,291.66,354.17


In [ ]:
creditos = pd.DataFrame({
    "id_customer": ["A", "B", "A"],
    "id_credit":[1001, 1002, 1003]
    "duration": [4,3,5],
    "date": ["2022-01-02", "2022-02-03", "2022-03-04"],
    "credit":[100, 90, 250]
    "installment": [25,30,50],
})

In [43]:
creditos = pd.DataFrame({
    "client_id": ["Juan", "Juan", "Ana", "Ana", "Pedro", "Pedro"],
    "credit": [1000, 5000, 2000, 3000, 1500, 2000],
    "installment": [12, 24, 6, 12, 8, 12],
    "date": ["2022-01-01", "2022-02-01", "2022-01-01", "2022-03-01", "2022-01-01", "2022-02-01"],
    "end_date": ["2023-01-01", "2024-02-01", "2022-07-01", "2023-03-01", "2022-09-01", "2023-02-01"]
})

creditos['installment_amount'] = round(creditos['credit'] / creditos['installment'],2)
creditos["date"] = pd.to_datetime(creditos["date"])
creditos["end_date"] = pd.to_datetime(creditos["end_date"])
creditos['month'] = creditos['date'].dt.to_period('M')
creditos['duration'] = ((creditos['end_date'] - creditos['date']) / np.timedelta64(1, 'M')).astype(int)
creditos['remaining_duration'] = creditos['duration'] - creditos['installment']
creditos.head()

,client_id,credit,installment,date,end_date,installment_amount,month,duration,remaining_duration
0,Juan,1000,12,2022-01-01,2023-01-01,83.33,2022-01,11,-1
1,Juan,5000,24,2022-02-01,2024-02-01,208.33,2022-02,23,-1
2,Ana,2000,6,2022-01-01,2022-07-01,333.33,2022-01,5,-1
3,Ana,3000,12,2022-03-01,2023-03-01,250.00,2022-03,11,-1
4,Pedro,1500,8,2022-01-01,2022-09-01,187.50,2022-01,7,-1


In [44]:
principal = 100000
interest_rate = 0.05
term_years = 5
payments_year = 12


In [45]:
term_months = term_years * 12
payments = term_months * payments_year

In [48]:
interest_rate_monthly = interest_rate / payments_year

In [52]:
monthly_payment = npf.pmt(interest_rate_monthly, payments, principal)
monthly_payment

-438.6418998606706

In [55]:
data = pd.DataFrame(columns=['Month', 'Payment', 'Interest', 'Principal', 'Balance'])
data.loc[0] = [0, 0, 0, 0, principal]

for i in range(1, payments+1):
    interest = data.loc[i-1, 'Balance'] * interest_rate_monthly
    principal = monthly_payment - interest
    balance = data.loc[i-1, 'Balance'] - principal
    data.loc[i] = [i, monthly_payment, interest, principal, balance]

data.head()

,Month,Payment,Interest,Principal,Balance
0,0.0,0.0000,0.000000,0.000000,100000.000000
1,1.0,-438.6419,416.666667,-855.308567,100855.308567
2,2.0,-438.6419,420.230452,-858.872352,101714.180919
3,3.0,-438.6419,423.809087,-862.450987,102576.631906
4,4.0,-438.6419,427.402633,-866.044533,103442.676439


In [56]:
data.tail()

,Month,Payment,Interest,Principal,Balance
716,716.0,-438.6419,16282.673514,-16721.315414,3.924563e+06
717,717.0,-438.6419,16352.345662,-16790.987562,3.941354e+06
718,718.0,-438.6419,16422.308110,-16860.950010,3.958215e+06
719,719.0,-438.6419,16492.562069,-16931.203968,3.975146e+06
720,720.0,-438.6419,16563.108752,-17001.750652,3.992148e+06
